# Build Simple Tool-Use AI Agents in LangGraph

## Learning Objectives
By the end of this notebook, you will be able to:
1. **Understand the Agent pattern** - Learn the key difference between augmented LLMs and agents
2. **Implement feedback loops** - Connect tool outputs back to the LLM for processing
3. **Build a complete AI Agent** - Create a system that can reason, act, and respond
4. **Understand the ReAct pattern** - See Reasoning + Acting in practice

## The Key Difference: Augmented LLM vs. AI Agent

### Augmented LLM (Previous Notebook)
```
User → LLM → Tool → END (raw tool output)
```
**Problem**: Tool results go directly to the user without LLM processing!

### AI Agent (This Notebook)
```
User → LLM → Tool → LLM → (maybe more tools) → Human-readable response
```
**Solution**: Feedback loop lets LLM process tool results!

## What Makes an Agent "Agentic"?

| Feature | Description |
|---------|-------------|
| **Autonomy** | Makes decisions without human intervention |
| **Reasoning** | Analyzes context to decide next action |
| **Tool Use** | Calls external tools when needed |
| **Feedback Loop** | Processes results and can take further actions |
| **Goal-Oriented** | Works toward completing user's objective |

### Tool-based Agentic AI System

- **Dynamic Decision-Making**: LLM determines whether to directly respond or invoke a tool based on the query context.
- **Seamless Tool Integration**: External tools are integrated to handle specific tasks, such as real-time web queries or computations.
- **Workflow Flexibility**: Conditional routing ensures efficient task delegation:
  - Tool Required: Routes to tool execution.
  - No Tool Required: Ends the workflow with an LLM response.
- **Feedback Loop**: Incorporates a feedback loop to improve responses by combining LLM insights and tool outputs to further improve responses or call more tools if needed

![](https://i.imgur.com/DHxiOLl.png)


In [ ]:
# ============================================================================
# SETUP: Import dependencies and LLM helper functions
# ============================================================================
import os
import sys

# Add parent directory to path for importing helpers
# This allows us to import utility functions from the parent directory
sys.path.append(os.path.abspath(".."))

# Import LLM factory functions
# These provide easy access to different LLM providers (Groq, OpenAI)
from helpers.utils import get_groq_llm, get_openai_llm

print("Setup complete! Ready to build an AI Agent.")

## Step 1: Define the Agent State

For an AI Agent with feedback loops, the message history becomes even more important:

### Message Flow in an Agent
```
1. HumanMessage     - User's original query
2. AIMessage        - LLM's response (with tool_calls)
3. ToolMessage      - Results from tool execution
4. AIMessage        - LLM processes tool results (may call more tools)
5. ToolMessage      - More tool results (if needed)
6. AIMessage        - Final human-readable response
```

The `add_messages` reducer preserves this entire chain, allowing the LLM to see:
- What the user asked
- What tools were called
- What results came back
- The full reasoning chain

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [ ]:
# ============================================================================
# DEFINING AGENT STATE
# ============================================================================
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    State schema for the AI Agent.
    
    The messages list tracks the complete interaction:
    - HumanMessage: User queries
    - AIMessage: LLM responses (may include tool_calls)
    - ToolMessage: Tool execution results
    
    The add_messages reducer ensures ALL messages are preserved,
    which is crucial for the feedback loop where the LLM needs
    to see what tools returned to generate a final response.
    
    This state structure supports the agentic pattern where:
    1. LLM can see the full conversation history
    2. Tool results are added to the state
    3. LLM can process tool results in subsequent calls
    4. Multiple tool calls can be chained together
    """
    messages: Annotated[list, add_messages]

## Step 2: Create Tools and Augment the LLM

Same as before - we create tools and bind them to the LLM.
The difference comes in how we **structure the graph** to create a feedback loop.

### Tool Creation Recap
1. Define Python function with `@tool` decorator
2. Write clear docstring (helps LLM decide when to use it)
3. Add type hints for parameters
4. Bind tools to LLM with `bind_tools()`

Here we define our custom search tool and then bind it to the LLM to augment the LLM.

In [ ]:
# ============================================================================
# CREATE TOOLS AND AUGMENT THE LLM
# ============================================================================
from langchain_openai import ChatOpenAI
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool

# -----------------------------------------------------------------------------
# Step 1: Initialize the base LLM
# We use GPT-4o for better reasoning and tool-use capabilities
# Temperature=0 makes responses more deterministic
# -----------------------------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print(f"LLM initialized: {llm.model_name}")

# -----------------------------------------------------------------------------
# Step 2: Create the web search tool using Tavily
# Tavily is a search API designed specifically for AI applications
# This tool allows the agent to fetch real-time information from the internet
# -----------------------------------------------------------------------------
tavily_search = TavilySearchAPIWrapper()

@tool
def search_web(query: str, num_results: int = 5):
    """
    Search the web for a query. Useful for general information or general news.
    
    This docstring is CRITICAL - it tells the LLM:
    - WHAT the tool does (search the web)
    - WHEN to use it (for current events, real-time info, facts beyond training data)
    
    Use this tool when:
    - The user asks about current events or news
    - The user needs real-time information
    - The question requires facts beyond your training data
    
    Args:
        query: The search query to look up
        num_results: Number of results to return (default 5)
        
    Returns:
        Search results containing titles, URLs, and content snippets
    """
    print(f"  [TOOL CALL] Searching web for: '{query}'")
    results = tavily_search.raw_results(
        query=query,
        max_results=num_results,
        search_depth='advanced',
        include_raw_content=True
    )
    return results

# -----------------------------------------------------------------------------
# Step 3: Create the tools list and bind to LLM
# bind_tools() creates an "augmented" LLM that knows about these tools
# This LLM can now decide when to call tools based on the user's query
# -----------------------------------------------------------------------------
tools = [search_web]  # List of all available tools

# Bind tools to the LLM
# This creates a new LLM that can decide to call tools when needed
llm_with_tools = llm.bind_tools(tools=tools)

print(f"Agent equipped with {len(tools)} tool(s): {[t.name for t in tools]}")

In [ ]:
# ============================================================================
# TEST: Verify the augmented LLM can make tool calls
# ============================================================================
# Let's test that our LLM is properly augmented and can make tool calls
# For queries requiring current information, the LLM should return tool_calls

print("Testing augmented LLM with tool-calling capability:")
print("-" * 50)
print("Query: 'what is the latest news on nvidia'")
print("Expected: LLM should return tool_calls (not direct answer)\n")

response = llm_with_tools.invoke('what is the latest news on nvidia')

# Check if the LLM called any tools
if response.tool_calls:
    print("✓ LLM correctly decided to call tools!")
    print(f"Tool calls: {response.tool_calls}")
else:
    print("Note: LLM answered directly (no tool calls needed)")
    print(f"Response: {response.content}")

## Step 3: Create the Graph with the Tool-Use Agentic System

### The Critical Difference: The Feedback Loop

**This is where we add the feedback loop that makes it an Agent!**

In the previous notebook (Augmented LLM), tool results went directly to END:
```
LLM → Tool → END (raw results)
```

In this notebook (AI Agent), we add a feedback loop:
```
LLM → Tool → LLM (processes results) → END or more tools
```

### Key Components

| Component | Purpose |
|-----------|---------|
| **tool_calling_llm** | Node that calls the augmented LLM |
| **tools** (ToolNode) | Executes tool calls |
| **tools_condition** | Routes based on whether LLM made tool calls |
| **Feedback loop** | Routes tool results back to LLM |

### The Agent Flow

```
┌─────────────────┐
│  START          │
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│tool_calling_llm │ ← LLM analyzes query
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│tools_condition  │ ← Check for tool_calls
└────────┬────────┘
    ┌────┴────┐
    │         │
    ▼         ▼
┌────────┐  ┌─────┐
│ tools  │  │ END │ (no tools needed)
└───┬────┘  └─────┘
    │
    │ [FEEDBACK LOOP] ← Tool results go back to LLM!
    │
    ▼
┌─────────────────┐
│tool_calling_llm │ ← LLM processes tool results
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│tools_condition  │ ← Check again
└────────┬────────┘
    ┌────┴────┐
    │         │
    ▼         ▼
┌────────┐  ┌─────┐
│ tools  │  │ END │ (final response)
└────────┘  └─────┘
```

**Note**: The feedback loop allows:
1. LLM to process tool results
2. Generate human-readable responses
3. Make additional tool calls if needed
4. Chain multiple tool calls together

![](https://i.imgur.com/DHxiOLl.png)

In [ ]:
# ============================================================================
# BUILD THE TOOL-USE AI AGENT GRAPH
# ============================================================================
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

# -----------------------------------------------------------------------------
# Step 1: Define the LLM node with tool-calling capability
# This node calls our tool-augmented LLM
# -----------------------------------------------------------------------------
def tool_calling_llm(state: State) -> State:
    """
    Node that calls the tool-augmented LLM.
    
    The LLM will:
    1. Analyze all messages in the state (including tool results)
    2. Decide if tools are needed or if it can respond directly
    3. Return either:
       - A direct response (AIMessage with content)
       - A tool call request (AIMessage with tool_calls)
    
    Note: This node is called multiple times in the feedback loop:
    - First: Processes user query
    - After tools: Processes tool results (this is the feedback!)
    """
    current_state = state["messages"]
    response = llm_with_tools.invoke(current_state)
    return {"messages": [response]}

# -----------------------------------------------------------------------------
# Step 2: Build the graph structure
# -----------------------------------------------------------------------------
builder = StateGraph(State)

# Add the LLM node (makes decisions about tool use)
builder.add_node("tool_calling_llm", tool_calling_llm)

# Add the ToolNode (executes tools when called)
# ToolNode is a pre-built node that:
# - Extracts tool calls from the AIMessage
# - Executes each tool with provided arguments
# - Returns ToolMessage(s) with results
builder.add_node("tools", ToolNode(tools=tools))

# Entry point: Start with the LLM
builder.add_edge(START, "tool_calling_llm")

# -----------------------------------------------------------------------------
# Step 3: Add conditional routing based on tool_calls
# tools_condition is a pre-built function that:
# - Returns "tools" if the LLM response has tool_calls
# - Returns END if no tool_calls (direct response)
# -----------------------------------------------------------------------------
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,  # Built-in routing: checks for tool_calls in response
    ["tools", END]    # Possible destinations
)

# -----------------------------------------------------------------------------
# Step 4: THE KEY FEEDBACK LOOP
# After tools execute, route results back to the LLM
# This allows the LLM to process tool results and generate a final response
# -----------------------------------------------------------------------------
builder.add_edge("tools", "tool_calling_llm")  # ← This is the feedback loop!

# Also add an edge to END from tools (for termination if needed)
# Note: This creates a potential cycle, but tools_condition will handle termination
builder.add_edge("tools", END)

# -----------------------------------------------------------------------------
# Step 5: Compile the graph
# -----------------------------------------------------------------------------
agent = builder.compile()

print("AI Agent graph compiled!")
print("Key feature: Feedback loop from tools → tool_calling_llm")
print("\nGraph structure:")
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
# ============================================================================
# VISUALIZE THE AGENT GRAPH
# ============================================================================
# Notice the key difference from the augmented LLM:
# There's a feedback loop from "tools" back to "tool_calling_llm"
# This allows the LLM to process tool results!

print("Agent Graph Visualization:")
print("-" * 50)
print("Flow options:")
print("  1. START → tool_calling_llm → END (no tools needed)")
print("  2. START → tool_calling_llm → tools → tool_calling_llm → END")
print("     (tools called, then LLM processes results)")
print("  3. START → tool_calling_llm → tools → tool_calling_llm → tools → ...")
print("     (multiple tool calls if needed)")
print("-" * 50)

agent  # Display the agent object

In [ ]:
# ============================================================================
# TEST 1: Query that DOESN'T need tools
# ============================================================================
# Simple factual question - LLM can answer from training data
# Expected: Direct response, no tool calls, single pass through graph

print("=" * 70)
print("TEST 1: Simple query (no tools needed)")
print("Query: 'Explain AI in 2 bullets'")
print("=" * 70)

user_input = "Explain AI in 2 bullets"

print("\nStreaming agent execution:")
print("Expected flow: START → tool_calling_llm → END")
print("-" * 70)

for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

print("\n" + "=" * 70)
print("Observation: Agent answered directly without calling any tools")
print("This is the same as an augmented LLM - no feedback loop needed!")
print("=" * 70)

In [ ]:
# ============================================================================
# TEST 2: Query that NEEDS tools (current information)
# ============================================================================
# This question requires current/real-time information
# Expected flow: 
#   1. LLM calls search_web tool
#   2. Tool returns results
#   3. LLM processes tool results (feedback loop!)
#   4. LLM generates human-readable final response

print("=" * 70)
print("TEST 2: Query requiring current information (tools needed)")
print("Query: 'What is the latest news on OpenAI product releases'")
print("=" * 70)

user_input = "What is the latest news on OpenAI product releases"

print("\nStreaming agent execution:")
print("Expected flow: START → tool_calling_llm → tools → tool_calling_llm → END")
print("Notice: Two LLM calls! (initial decision + processing tool results)")
print("-" * 70)

for event in agent.stream({"messages": user_input},
                          stream_mode='values'):
    event['messages'][-1].pretty_print()

print("\n" + "=" * 70)
print("Observation: Agent used the feedback loop!")
print("1. LLM decided to call search_web tool")
print("2. Tool executed and returned results")
print("3. LLM processed tool results and generated human-readable response")
print("This is the key difference from an augmented LLM - the agent processes tool results!")
print("=" * 70)

In [ ]:
# ============================================================================
# EXPLORE: Examine the final message
# ============================================================================
# The final message should be an AIMessage with a human-readable response
# that incorporates the tool results

print("Final message in the conversation:")
print("-" * 70)
event['messages'][-1]

In [ ]:
# ============================================================================
# DISPLAY: View the final human-readable response
# ============================================================================
# This shows the agent's final response after processing tool results
# Compare this to the raw tool output - the agent has synthesized it into
# a coherent, human-readable answer

from IPython.display import display, Markdown

print("Final Agent Response (formatted as Markdown):")
print("=" * 70)
display(Markdown(event['messages'][-1].content))
print("\n" + "=" * 70)
print("Note: This is a synthesized response based on tool results,")
print("not just the raw tool output!")
print("=" * 70)

## Summary: Key Takeaways

### What We Built
A complete **AI Agent** that can:
- Decide when to use external tools
- Execute web searches for current information
- **Process tool results** (the key difference!)
- Generate human-readable responses from tool outputs
- Chain multiple tool calls if needed

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Feedback Loop** | Routes tool results back to LLM for processing |
| **Agent vs Augmented LLM** | Agents process tool results; augmented LLMs just execute tools |
| **ToolNode** | Pre-built node that executes tool calls |
| **tools_condition** | Pre-built routing based on tool_calls |

### The Agent Pattern

```
1. User Query → LLM
2. LLM decides: Answer directly OR call tools
3a. Direct answer → Return response → END
3b. Call tools → ToolNode executes → Results back to LLM (feedback loop!)
4. LLM processes tool results → Generate human-readable response → END
```

### Why This Matters

**Without feedback loop (Augmented LLM):**
- Tool results go directly to user
- Raw, unprocessed output
- No synthesis or reasoning

**With feedback loop (AI Agent):**
- LLM processes tool results
- Generates coherent, synthesized responses
- Can make additional tool calls based on results
- True agentic behavior!

### What's Next?

You've now built a complete AI Agent! Next steps could include:
- Adding multiple tools (calculator, database queries, etc.)
- Implementing memory/persistence for conversations
- Adding error handling and retry logic
- Building multi-agent systems
- Adding human-in-the-loop confirmation for critical actions